# Ex18



Considerar a produção das latas que são estampadas em folhas planas de alumínio. Uma lata consiste em um corpo principal e em duas extremidades (topo e base). Existem **4 padrões de estampas possíveis**, que envolvem **2 tamanhos diferentes** de folha de metal, uma com dimenções de 70x75 e outra com 65x105

onde,

| MATRIZ | Extremidades | Corpo | tamanho|
|--------------|------------|-------------|-------------|
| **1**     | 3     | 3      | 70x75|
| **2**   | 7      | 1      |70x75|
| **3**     | 5   | 4     | 65x105 |
| **4**     | 9      | 3      |70x75|



Considerando que deverão ser fabricadas 5 mil latas, qual a forma mais economica de produzí-las?






## **Variáveis de Decisão**
Definição das variáveis:
- $x_j$ : Número de folhas estampadas utilizando a matriz $j$, onde $j \in \{1, 2, 3, 4\}$.

## **Função Objetivo**
Minimizar o número total de folhas de alumínio utilizadas:

$$
\text{Minimize } Z = x_1 + x_2 + x_3 + x_4
$$

## **Restrições**

### **1. Produção mínima de extremidades**
Cada lata necessita de **2 extremidades** (topo e base), então para 5.000 latas, são necessários **10.000 extremidades**:

$$
3x_1 + 7x_2 + 5x_3 + 9x_4 \geq 10000
$$

### **2. Produção mínima de corpos**
Cada lata necessita de **1 corpo**, então para 5.000 latas, são necessários **5.000 corpos**:

$$
3x_1 + 1x_2 + 4x_3 + 3x_4 \geq 5000
$$

### **3. Restrição de Não-Negatividade**
As quantidades de folhas utilizadas devem ser **não negativas**:

$$
x_j \geq 0, \quad \forall j \in \{1, 2, 3, 4\}
$$




In [15]:
import pyomo.environ as pyo

In [16]:
# Criação do modelo
model = pyo.ConcreteModel()

In [17]:

# Conjuntos
model.Matrizes = pyo.Set(initialize=[1, 2, 3, 4])


In [18]:
# Coeficientes de produção de extremidades por matriz
coef_extremidades = {1: 3, 2: 7, 3: 5, 4: 9}
model.coef_ext = pyo.Param(model.Matrizes, initialize=coef_extremidades)

# Coeficientes de produção de corpos por matriz
coef_corpos = {1: 3, 2: 1, 3: 4, 4: 3}
model.coef_corpo = pyo.Param(model.Matrizes, initialize=coef_corpos)

# Demanda mínima
demanda_extremidades = 10000
demanda_corpos = 5000

In [19]:

# ------------------------------
# Variáveis de Decisão
# ------------------------------

model.x = pyo.Var(model.Matrizes, domain=pyo.NonNegativeIntegers)

In [20]:

# ------------------------------
# Função Objetivo
# ------------------------------
def fo(model):
    return sum(model.x[j] for j in model.Matrizes)

model.objetivo = pyo.Objective(rule=fo, sense=pyo.minimize)


In [21]:
# ------------------------------
# Restrições
# ------------------------------

# 1. Produção mínima de extremidades
def restricao_extremidades(model):
    return sum(model.coef_ext[j] * model.x[j] for j in model.Matrizes) >= demanda_extremidades

model.restricao_extremidades = pyo.Constraint(rule=restricao_extremidades)

# 2. Produção mínima de corpos
def restricao_corpos(model):
    return sum(model.coef_corpo[j] * model.x[j] for j in model.Matrizes) >= demanda_corpos

model.restricao_corpos = pyo.Constraint(rule=restricao_corpos)

In [22]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex18.lp", io_options={"symbolic_solver_labels": True})

('ex18.lp', 2379369791344)

In [24]:
# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)
valor_obj = pyo.value(model.objetivo)
formatted = f"{valor_obj:,.2f}"
formatted = formatted.replace(",", "X").replace(".", ",").replace("X", ".")
print("Valor da Função Objetivo: ", formatted)

MIP  has 2 rows; 4 cols; 8 nonzeros; 4 integer variables (0 binary)
Coefficient ranges:
  Matrix [1e+00, 9e+00]
  Cost   [1e+00, 1e+00]
  Bound  [0e+00, 0e+00]
  RHS    [5e+03, 1e+04]
Presolving model
2 rows, 4 cols, 8 nonzeros  0s
2 rows, 4 cols, 8 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   2 rows
   4 cols (0 binary, 4 integer, 0 implied int., 0 continuous)
   8 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point; X => User solution

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00% 

In [25]:

for j in model.Matrizes:
    valor = pyo.value(model.x[j])
    print(f"Matriz {j}: {valor:.2f} chapas")

Matriz 1: 0.00 chapas
Matriz 2: 0.00 chapas
Matriz 3: 714.00 chapas
Matriz 4: 715.00 chapas
